In [ ]:
import numpy as np
import pandas as pd
import SAGA
import matplotlib.pyplot as plt
import seaborn as sns
from operator import itemgetter
import os
from dotenv import load_dotenv

from SAGA import ObjectCuts as C

load_dotenv()

In [ ]:
# Data manipulations

def get_radii(coordinates):
    radii = []
    for coordinate_pair in coordinates:
        radii.append(np.sqrt(coordinate_pair[0]**2+coordinate_pair[1]**2))
    return radii
    
def get_ellipticities(coordinates):
    # # First get quadrupole moments
    coordinates_squared = np.square(coordinates)

    xdiff_ydiff_col = np.prod(coordinates, axis=2)
    xdiff_ydiff_col = xdiff_ydiff_col[:,:,np.newaxis]

    quadrupole_moments_setup = np.insert(coordinates_squared, 0, [[[xdiff_ydiff_col]]], axis=2) # diffx*diffy now at position 0
    quadrupole_moments = quadrupole_moments_setup.mean(axis=1)

    # # Get ellipticity from quadrupole moments
    # Q_xy, Q_xx, Q_yy = np.quadrupole_moments
    quadrupole_xx_yy = quadrupole_moments[:,1:3]
    quadrupole_xy = quadrupole_moments[:,0:1]

    e_1_numerator = -np.diff(quadrupole_xx_yy, axis=1)
    e_2_numerator = 2*quadrupole_xy

    e_denominator_1 = np.sum(quadrupole_xx_yy, axis=1)[:,np.newaxis]
    e_denominator_inside_sqrt = np.prod(quadrupole_xx_yy, axis=1)[:,np.newaxis] - quadrupole_xy**2
    e_denominator = e_denominator_1 + 2*np.sqrt(e_denominator_inside_sqrt)

    e_1 = e_1_numerator / e_denominator
    e_2 = e_2_numerator / e_denominator

    ellipticities = np.sqrt(e_1**2 + e_2**2)[:,0]

    return ellipticities


def get_randomized_ellipticities(radii, num_repititions):
    num = len(radii)
    radial_stdev = np.std(radii)

    # Create "isotropic" (random from uniform - not uniform) angles
    coords = np.random.randn(num_repititions, num, 2)  # (number of times to repeat data set generation, how many satellites, how many spatial dimensions)
    calc_radii = np.sqrt((coords**2).sum(axis=2))
    normalized_coords = coords/calc_radii[:,:,np.newaxis]

    # Move the positions of satellites out or in radially until they reach original radial scale
    reshaped_radii = np.reshape(radii, (1,len(radii), 1))
    scaled_coords = normalized_coords*reshaped_radii

    ellipticities = get_ellipticities(scaled_coords,)
    
    return ellipticities


def get_prominence(actual, radii, num_repititions):
    randomized = get_randomized_ellipticities(radii, num_repititions)
    abv_actual = np.where(randomized > actual, 1, 0)
    p_above_actual = np.sum(abv_actual)/num_repititions
    prominence = 1/p_above_actual
    
    return prominence

In [ ]:
# Dataframe manipulations
def filter_subhalo_dataframe_by_num_satellites(cutoff):
    filtered_dataframe = VSMDPL_subs_raw[VSMDPL_subs_raw['vmax_mpeak'] >= cutoff].copy()
    filtered_dataframe = filtered_dataframe.groupby(['HOSTID']).filter(lambda x: len(x) > 2).copy()
    return filtered_dataframe

def add_coordinate_columns(dataframe):
    dataframe['x_coord'] = dataframe['x_adj'] - dataframe['x_host']
    dataframe['y_coord'] = dataframe['y_adj'] - dataframe['y_host']
    # dataframe['z_coord'] = dataframe['z_adj'] - dataframe['z_host']
    # dataframe['3d_coordinates'] = dataframe[['x_coord', 'y_coord', 'z_coord']].values.tolist()
    dataframe['2d_coordinates'] = dataframe[['x_coord', 'y_coord']].values.tolist()

def get_systems_dataframe(dataframe):
    systems_df = dataframe.groupby('HOSTID')['2d_coordinates'].apply(tuple).reset_index(name='coordinates_list')
    systems_df['coordinates_list'] = systems_df['coordinates_list'].tolist()
    systems_df['radii'] = systems_df['coordinates_list'].apply(get_radii)
    systems_df['Dataset'] = 'Actual'
    return systems_df

def add_ellipticity_column(dataframe):
    dataframe['ellipticity'] = dataframe.apply(lambda x: get_ellipticities([x['coordinates_list']])[0], axis=1)

def get_isotropic_data(dataframe):
    dataframe_copy = dataframe.copy()
    dataframe_copy['ellipticity'] = dataframe_copy.apply(lambda x: get_randomized_ellipticities(x['radii'], 1)[0], axis=1)
    dataframe_copy['Dataset'] = 'Isotropic'
    return dataframe_copy

def add_num_subs_column(dataframe):
    dataframe['num_subs'] = dataframe['radii'].str.len()
    display(dataframe)

def add_prominence_column(dataframe, num_iterations):
    dataframe['prominence'] = dataframe.apply(lambda x: get_prominence(x['ellipticity'], x['radii'], num_iterations), axis=1)

def get_prominence_dataframe(dataframe):
    prominence_data = dataframe[['HOSTID', 'ellipticity', 'prominence', 'Dataset']].copy()
    return prominence_data

def plot_prominence(data, num_iterations):
    format = {
        "figure.facecolor": "212946",
        "axes.facecolor": "212946",
        "savefig.facecolor": "212946", 
        "grid.color": "2A3459",
        "text.color": "0.9",
        "axes.labelcolor": "0.9",
        "xtick.color": "0.9",
        "ytick.color": "0.9",
        "grid.linestyle": "-",
        "lines.solid_capstyle": "round",
        "figure.figsize" : "(11.7,8.27)"
    }

    sns.set_style("darkgrid", format)

    fig, ax1 = plt.subplots()
    sp1 = sns.ecdfplot(data=data, x="prominence", hue="Dataset", hue_order=['Actual', 'Isotropic'], complementary=True, ax=ax1)
    ax1.set_xscale('log')
    ax1.set_xlim(1,num_iterations)
    plt.show()

In [ ]:
def get_ellipticity_and_prominence_data(dataframe, num_iterations, num_isotropic):
    add_coordinate_columns(dataframe)
    systems_dataframe = get_systems_dataframe(dataframe).copy()
    add_ellipticity_column(systems_dataframe) # this is the original dataframe
    add_prominence_column(systems_dataframe, num_iterations) # bottleneckyy
    prominence_data = get_prominence_dataframe(systems_dataframe).copy()

    for i in range (0,num_isotropic):
        systems_dataframe_isotropic = get_isotropic_data(systems_dataframe).copy()
        add_prominence_column(systems_dataframe_isotropic, num_iterations)  
        prominence_data_isotropic = get_prominence_dataframe(systems_dataframe_isotropic).copy()
        prominence_data = pd.concat([prominence_data, prominence_data_isotropic], ignore_index=True)

    return [systems_dataframe, prominence_data]

## Get Data

In [ ]:
HOST_PROPERTIES = os.getenv('HOST_PROPERTIES')
saga_hosts = pd.read_parquet(HOST_PROPERTIES)

# from SAGA-halo-ellipticities (filtered + cartesian coords)
%store -r

iterations = 10000

# Apples to apples these bad boys
VSMDPL_subs_raw.rename(columns = {'upid':'HOSTID'}, inplace = True)
saga_sats.rename(columns = {'X':'x_adj', 'Y':'y_adj', 'HOST_X':'x_host', 'HOST_Y':'y_host'}, inplace = True)
VSMDPL_subs_interlopers.rename(columns = {'interloper_host_id':'HOSTID'}, inplace = True)

# Use vmax_mpeak as proxy for luminosity
vmax_cutoff_for_avg_num = {
    '3.5' : 61,
      '4' : 51.6,
    '4.5' : 46.4,
      '5' : 43.5,
    '5.5' : 40.98,
    'saga': 44.09 # 4.855 sats per host on average
}

VSMDPL_subs_multi = {}
for cutoff in list(vmax_cutoff_for_avg_num.keys())[:5]:
    VSMDPL_subs_multi[cutoff] = filter_subhalo_dataframe_by_num_satellites(vmax_cutoff_for_avg_num[cutoff])
VSMDPL_subs = filter_subhalo_dataframe_by_num_satellites(vmax_cutoff_for_avg_num['saga'])
saga_sats = saga_sats.groupby(['HOSTID']).filter(lambda x: len(x) > 2).copy()


VSMDPL_subs_interlopers

## SAGA Analysis

In [ ]:
saga_data, saga_plot = get_ellipticity_and_prominence_data(saga_sats, iterations, 1)
plot_prominence(saga_plot, iterations)

# saga_data

#### Generate 100x isotropic data

In [ ]:
saga_data, saga_plot = get_ellipticity_and_prominence_data(saga_sats, iterations, 100)
plot_prominence(saga_plot, iterations)
# saga_data

## SIM Analysis

In [ ]:
VSMDPL_data, VSMDPL_plot = get_ellipticity_and_prominence_data(VSMDPL_subs, iterations, 1)
plot_prominence(VSMDPL_plot, iterations)
# VSMDPL_data

## SAGA + SIM

In [ ]:
saga_plot['Datasource'] = 'Saga'
VSMDPL_plot['Datasource'] = 'VSMDPL'
saga_sim_data = pd.concat([saga_plot, VSMDPL_plot], ignore_index=True)

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=saga_sim_data, x="prominence", hue=saga_sim_data[['Datasource', 'Dataset']].apply(tuple, axis=1), complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_xlim(1,10000)

In [ ]:
saga_data['Datasource'] = 'Saga'
VSMDPL_data['Datasource'] = 'VSMDPL'
saga_sim_ellipticity_data = pd.concat([saga_data, VSMDPL_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ more spherical         Ellipticity (2D)         more elliptical $\longrightarrow$')
sp1 = sns.histplot(saga_sim_ellipticity_data, x="ellipticity", hue="Datasource", hue_order=['Saga', 'VSMDPL'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='viridis')

## SIM Compare prominence for different avg # satellites

In [ ]:
VSMDPL_systems_multi = {}
VSMDPL_plot_multi = {}

for avg in list(vmax_cutoff_for_avg_num.keys())[:5]:
    VSMDPL_systems_multi[avg], VSMDPL_plot_multi[avg] = get_ellipticity_and_prominence_data(VSMDPL_subs_multi[avg], iterations, 1)
    VSMDPL_plot_multi[avg]['Average # Satellites'] = avg
    VSMDPL_systems_multi[avg]['Datasource'] = 'VSMDPL ' + avg

In [ ]:
VSMDPL_multi_plot_data = pd.concat(VSMDPL_plot_multi, ignore_index=True)
VSMDPL_multi_plot_data = VSMDPL_multi_plot_data[VSMDPL_multi_plot_data['Dataset'] == 'Actual'].copy()

fig, ax1 = plt.subplots()
sp1 = sns.ecdfplot(data=VSMDPL_multi_plot_data, x="prominence", hue='Average # Satellites', hue_order=['3.5', '4', '4.5', '5', '5.5'], complementary=True, ax=ax1)
ax1.set_xscale('log')
ax1.set_xlim(1,iterations)

In [ ]:
for df in VSMDPL_systems_multi.values():
    add_num_subs_column(df)

In [ ]:
VSMDPL_multi_data = pd.concat(VSMDPL_systems_multi, ignore_index=True)
fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ more spherical         Ellipticity (2D)         more elliptical $\longrightarrow$')
sp1 = sns.histplot(VSMDPL_multi_data, x="ellipticity", hue="Datasource", hue_order=['VSMDPL 3.5', 'VSMDPL 4', 'VSMDPL 4.5', 'VSMDPL 5', 'VSMDPL 5.5'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='viridis')

In [ ]:
all_data = pd.concat([VSMDPL_multi_data, saga_sim_ellipticity_data], ignore_index=True)

fig, ax1 = plt.subplots()
ax1.set_xlabel('$\longleftarrow$ more spherical         Ellipticity (2D)         more elliptical $\longrightarrow$')
sp1 = sns.histplot(all_data, x="ellipticity", hue="Datasource", hue_order=['Saga', 'VSMDPL', 'VSMDPL 3.5', 'VSMDPL 4', 'VSMDPL 4.5', 'VSMDPL 5', 'VSMDPL 5.5'], bins=7, element="poly", stat="density", common_norm=False, common_bins=True, fill=False, ax=ax1, palette='viridis')

### VSMDPL Interloper cuts

In [ ]:
VSMDPL_interloper_data, VSMDPL_interloper_plot = get_ellipticity_and_prominence_data(VSMDPL_subs_interlopers, iterations, 1)
plot_prominence(VSMDPL_interloper_plot, iterations)
# VSMDPL_interloper_data

## Ellipticity and Prominence vs Host properties

In [ ]:
add_num_subs_column(saga_data)
saga = saga_hosts.merge(saga_data, on="HOSTID")
saga = saga.rename(columns={"num_subs":"num_sats"})
saga

In [ ]:
saga['num_sats'] = saga['num_sats'].astype(np.float64)
saga


In [ ]:
# sns.reset_defaults()
sns.scatterplot(saga, x='ellipticity', y='prominence', edgecolor=None, hue=saga['num_sats'], palette='cividis', legend='full')
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
# sns.reset_defaults()
isotropic_saga_data = saga_plot[saga_plot['Dataset']=='Isotropic'].copy()
isotropic_saga_data = isotropic_saga_data.merge(saga_data[['HOSTID', 'radii']], on='HOSTID').copy()
add_num_subs_column(isotropic_saga_data)
isotropic_saga_data = isotropic_saga_data.rename(columns={"num_subs":"num_sats"})


sns.scatterplot(isotropic_saga_data, x='ellipticity', y='prominence', edgecolor=None, hue=isotropic_saga_data['num_sats'], palette='viridis', legend='full', size=1)
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
saga_copy = saga[['HOSTID', 'ellipticity', 'prominence', 'Dataset', 'radii', 'num_sats']]
raw_and_isotropic_saga_data = pd.concat([saga_copy, isotropic_saga_data])
# raw_and_isotropic_saga_data[raw_and_isotropic_saga_data['Dataset'] == 'Actual']['edgecolor'] = 'white'
raw_and_isotropic_saga_data.loc[raw_and_isotropic_saga_data["Dataset"] == "Actual", "edgecolor"] = 'white'
raw_and_isotropic_saga_data.loc[raw_and_isotropic_saga_data["Dataset"] == "Isotropic", "edgecolor"] = 'None'
raw_and_isotropic_saga_data


In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(raw_and_isotropic_saga_data[raw_and_isotropic_saga_data['Dataset']=='Isotropic'], x='ellipticity', y='prominence', hue='num_sats', edgecolor=None, palette='viridis', legend=None, ax=ax, size=0.1)
sns.scatterplot(raw_and_isotropic_saga_data[raw_and_isotropic_saga_data['Dataset']=='Actual'], x='ellipticity', y='prominence', hue='num_sats', edgecolor='white', palette='viridis', legend=None, ax=ax, linewidth=1)
plt.yscale('log')
plt.xlim([0,1])
plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="MORPHTYPE", y="ellipticity", hue='num_sats', edgecolor=None, palette='viridis', ax=axs[0])
sns.scatterplot(data=saga, x="radius", y="ellipticity", hue='num_sats', edgecolor=None, palette='viridis', ax=axs[1])
sns.scatterplot(data=saga, x="ba", y="ellipticity", hue='num_sats', edgecolor=None, palette='viridis', ax=axs[2])
sns.scatterplot(data=saga, x="phi", y="ellipticity", hue='num_sats', edgecolor=None, palette='viridis', ax=axs[3])

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="sb_r", y="ellipticity", ax=axs[0])
sns.scatterplot(data=saga, x="SERSIC", y="ellipticity", ax=axs[1])
sns.scatterplot(data=saga, x="K_ABS", y="ellipticity", ax=axs[2])
sns.scatterplot(data=saga, x="gr", y="ellipticity", ax=axs[3])

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="MORPHTYPE", y="prominence", ax=axs[0])
sns.scatterplot(data=saga, x="radius", y="prominence", ax=axs[1])
sns.scatterplot(data=saga, x="ba", y="prominence", ax=axs[2])
sns.scatterplot(data=saga, x="phi", y="prominence", ax=axs[3])
axs[1].set_xscale('log')
for ax in axs:
    ax.set_yscale('log')s

# next: show median prominence per morphology type (maybe group similar types together)

In [ ]:
fig, axs = plt.subplots(ncols=4)
fig.set_dpi(100)
fig.set_size_inches(30, 6, forward=True)
sns.scatterplot(data=saga, x="sb_r", y="prominence", ax=axs[0])
sns.scatterplot(data=saga, x="SERSIC", y="prominence",ax=axs[1])
sns.scatterplot(data=saga, x="K_ABS", y="prominence", ax=axs[2])
sns.scatterplot(data=saga, x="gr", y="prominence", ax=axs[3])
for ax in axs:
    ax.set_yscale('log')